In [2]:

import logging
from pathlib import Path

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from accelerate import Accelerator
import tensorflow as tf



2023-07-12 11:14:30.941728: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 11:14:31.734064: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from datasets import load_dataset

In [5]:
from torch.utils.data.dataloader import DataLoader

In [6]:
from torch.utils.data import IterableDataset

In [ ]:
class ConstantLengthDataset(IterableDataset):
    def __init__(self, tokenizer, dataset, field, seq_length=1024, num_of_sequences=1024, chars_per_token=3.6):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.bos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.input_characters = seq_length * chars_per_token * num_of_sequences
        self.field=field

    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.input_characters:
                    break
                try:
                    buffer.append(next(iterator)[self.field])
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    more_examples = False
                    break
            tokenized_inputs = self.tokenizer(buffer, truncation=False)["input_ids"]
            all_token_ids = []
            for tokenized_input in tokenized_inputs:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])
            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    yield torch.tensor(input_ids)

In [58]:
def create_dataloader(args,tokenizer):
    data_files  = {"test":args['test_bed_name']}
    valid_data = load_dataset(args['data_path'], data_files=data_files, split="test")
    valid_dataset = ConstantLengthDataset(tokenizer, valid_data, args['field'], seq_length=args['seq_length'])
    eval_dataloader = DataLoader(valid_dataset, batch_size=args['batch_size'])
    return  eval_dataloader

In [26]:
def evaluate(args,model,eval_dataloader):
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch, labels=batch)
        loss = outputs.loss.repeat(args['batch_size'])
        losses.append(accelerator.gather(loss))

        if args['max_eval_steps'] > 0 and step >= args['max_eval_steps']:
            break
    loss = torch.mean(torch.cat(losses))
    try:
        perplexity = torch.exp(loss)
    except OverflowError:
        perplexity = float("inf")
    return loss.item(), perplexity.item()




In [7]:
def param_default():
    model_name = 'codeparrot-small' #<-- Scope
    test_bed_name='code_completion_dataset_3k_deduped.json'
    semeru_datases_path= '/workspaces/code-rationales/'
    data_path = Path(semeru_datases_path+'datax/' + model_name + '/')
    data_path= semeru_datases_path+'semeru-datasets/semeru/galeras/galeras_se_tasks_dataset_3k_deduplicated'
    #data_path_raw = Path('../athena-datasets/' + corpus + '/raw/')
    #tokenizer_path = Path('../tokenizer/')
    return {
        'out_processed' : '/datasets/out_processed/',
        'checkpoint_file': Path(semeru_datases_path+'data/codeparrot-small/checkpoints/checkpoint-29000'), #Model
        'output_results' : 'results/' ,
        'seed': 1,
        'data_path': data_path,
        'test_bed_name':test_bed_name,
        'seq_length': 64,
        'batch_size': 2,
        'field': "random_cut",
        'max_eval_steps':-1
    }

In [8]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [9]:
device

device(type='cuda', index=0)

In [48]:
torch.cuda.memory_allocated()

480649216

In [ ]:
# Setup Accelerator
accelerator = Accelerator()
params = param_default()
# Parse configuration
set_seed(params['seed'])

# Logging
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO
)

# Load model and tokenizer
checkpoint = params['checkpoint_file']
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = model.to( device ) #WARNING, Verify the device before assigning to memory

# Load dataset and dataloader
valid_dataset, eval_dataloader = create_dataloader(params,tokenizer)

# Prepare everything with our `accelerator`.
model, valid_dataset, eval_dataloader = accelerator.prepare(model, valid_dataset, eval_dataloader)

# Evaluate and save the last checkpoint
logger.info("Evaluating and saving model after training")
eval_loss, perplexity = evaluate(params, model, eval_dataloader)
logger.info(f"loss/eval: {eval_loss}, perplexity: {perplexity}")

## Naive test for code completion

In [35]:
device ="cpu"

In [43]:
prompt ="def duntion_test():"
prompt="def test_frequency_condition_alone(self):\n        prev_hour = timezone.now() - timedelta(hours=1)"
params = param_default()

#torch.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(params['checkpoint_file'])
tokenizer = AutoTokenizer.from_pretrained(params['checkpoint_file'])
model = model.to( device ) #WARNING, Verify the device before assigning to memory
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(input_ids, do_sample=True, max_length=128)

tokenizer.batch_decode(outputs, skip_special_tokens=True)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['def test_frequency_condition_alone(self):\n        prev_hour = timezone.now() - timedelta(hours=1)\n        now = timezone.now() - timedelta(days=5, hours=30)\n        self.assertTrue(datetime.time())\n\n        now = timezone.now()\n        now1 = datetime.datetime.now().replace(hour=now)\n        with patch_timestamp(dt=datetime.now())\n\n        expected_before = date(1970, 1, 1)\n\n        # get all the microseconds given an exception with one day\n        self.assertEqual(now, datetime.now() - timedelta(']

## Outcome generation & Levenshtein evaluation

In [37]:
## This iterator is NOT working for batches > 1!!

class ConstantTokenLengthDataset(IterableDataset):
    def __init__(self, tokenizer, dataset, field, num_of_tokens=64, num_of_sequences=1024):
        self.tokenizer = tokenizer
        self.dataset = dataset
        self.num_of_tokens = min(num_of_tokens, tokenizer.model_max_length)
        self.field=field
        self.input_char = int(self.num_of_tokens*3.6)
        self.num_of_sequences=num_of_sequences
        self.prompts=[]

    def __iter__(self):  
        for i, buffer in enumerate(self.dataset):
            size = min(len(buffer[self.field]),self.input_char)
            input = buffer[self.field][:size]
            self.prompts.append(input)
            if i > self.num_of_sequences:
                break
        tokenized_inputs = self.tokenizer(self.prompts, max_length= self.num_of_tokens, padding=True, truncation=True, return_tensors="pt")["input_ids"]
        for tokenized_input in tokenized_inputs:
            yield torch.tensor(tokenized_input)

In [41]:
def create_dataloader(args,tokenizer):
    data_files  = {"test":args['test_bed_name']}
    valid_data = load_dataset(args['data_path'], data_files=data_files, split="test")
    valid_dataset = ConstantTokenLengthDataset(tokenizer, valid_data, args['field'], num_of_tokens=args['seq_length'])
    eval_dataloader = DataLoader(valid_dataset, batch_size=1)
    return  valid_dataset, eval_dataloader

In [42]:
def generate_outcomes(args,model,eval_dataloader,valid_data):
    model.eval()
    results = []
    for step, inputs in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model.generate(inputs, do_sample=True, max_length=128)
            outcome = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        prompt=valid_data.prompts[step]
        result = {"prompt": prompt, "outcome":outcome}
        results.append(result)
        if args['max_eval_steps'] > 0 and step >= args['max_eval_steps']:
            break
    return results


In [44]:
# Setup Accelerator
accelerator = Accelerator()
params = param_default()
# Parse configuration
set_seed(params['seed'])

# Logging
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO
)

# Load model and tokenizer
checkpoint = params['checkpoint_file']
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token
model = model.to( device ) #WARNING, Verify the device before assigning to memory

# Load dataset and dataloader
valid_dataset, eval_dataloader = create_dataloader(params,tokenizer)

# Prepare everything with our `accelerator`.
model, valid_dataset, eval_dataloader = accelerator.prepare(model, valid_dataset, eval_dataloader)

# Evaluate and save the last checkpoint
logger.info("Evaluating and saving model after training")
outcomes = generate_outcomes(params, model, eval_dataloader,valid_dataset)
logger.info(f"outomces: {len(outcomes)}")

07/12/2023 12:27:52 - WARNING - datasets.builder - Found cached dataset json (/root/.cache/huggingface/datasets/json/galeras_se_tasks_dataset_3k_deduplicated-f07ebc227c0463f3/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
07/12/2023 12:27:52 - INFO - __main__ - Evaluating and saving model after training
/tmp/ipykernel_2081683/994227097.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yield torch.tensor(tokenized_input)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token

In [ ]:
outcomes

[{'prompt': tensor([[318, 511,  63,  ...,   0,   0,   0],
          [  0, 318, 511,  ...,   0,   0,   0]], device='cuda:0'),
  'outcome': ['def test_frequency_condition_alone(self):\n        prev_hour = timezone.now() - timedelta(hours=1)\n        group = None\n        for i in range(5):\n            group = self.store_event(\n                project_id=self.project.id, data={"timestamp": iso_format(prev_hour)}\n            ).group\n        conditions = [\n            {\n    Corporation',
   'def test_expanding(data):\n    modin_series, _ = create_ License']},
 {'prompt': tensor([[  0,   0, 318,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]], device='cuda:0'),
  'outcome': ['def setup_method(self):\n        self.df = DataFrame({"A": [1, 2, 3]})\n        self.expected1 = self.df[self.df.A > 0]\n        self.expected2 = self.df.A + 1\n',
   "def test_chaining_upgraded_chords_mixed_canvas(self, manager, subtests):\n        \n        try:\n            manager.app.bac

In [12]:
import pandas as pd
import textdistance

In [17]:
levenshtein_similarity = textdistance.levenshtein


1024

lev_calc = [levenshtein_similarity.normalized_similarity(x["prompt"].strip(), x["outcome"]["predicted"].strip() ) for x in json_data]
